# Telluric Models

Similarly to stellar templates, the telluric model is constructed through a manager that will handle the construction and data storage.

The construction of the telluric model follows a recipe:

1) Construct a transmittance model from a given source (by default, Telfit)
2) Find the wavelength regions in which the transmittance model is smaller than a given percentage of the continuum
3) Account for the BERV by either using the +- Maximum yearly variation or by shifting the mask to all BERVS in our dataset.

The following workflow summarizes the main steps in this process:

![teste](../diagrams/telluric_template_construction.drawio.png)


In [1]:
from pathlib import Path

from ASTRA.data_objects import DataClassManager
from ASTRA.data_objects.DataClass import DataClass
from ASTRA.Instruments import ESPRESSO
from ASTRA.template_creation.TelluricModel import TelluricModel
from ASTRA.utils.choices import TELLURIC_CREATION_MODE, TELLURIC_EXTENSION

storage_path = Path("../../tmp")
manager = DataClassManager()
manager.start()

data: DataClass = manager.DataClass(
    list(Path("/home/amiguel/spectra_collection/ESPRESSO/proxima").glob("*.fits")),
    storage_path=storage_path,
    instrument=ESPRESSO,
)

ModelTell = TelluricModel(
    usage_mode="individual",
    user_configs={"CREATION_MODE": TELLURIC_CREATION_MODE.telfit,
                  "EXTENSION_MODE": TELLURIC_EXTENSION.LINES},
    root_folder_path=storage_path,
)

ModelTell.Generate_Model(
    dataClass=data,
    telluric_configs={"continuum_percentage_drop": 1},
    force_computation=False,
    store_templates=True,
)


2025-04-15 15:23:44.309 | DEBUG    | ASTRA.utils.UserConfigs:receive_user_inputs:216 - Generating internal configs of  - 
2025-04-15 15:23:44.314 | INFO     | ASTRA.utils.UserConfigs:receive_user_inputs:221 - Checking for any parameter that will take default value
2025-04-15 15:23:44.316 | DEBUG    | ASTRA.utils.UserConfigs:receive_user_inputs:228 - Configuration <SAVE_DISK_SPACE> using the default value: DISK_SAVE_MODE.DISABLED
2025-04-15 15:23:44.318 | DEBUG    | ASTRA.utils.UserConfigs:receive_user_inputs:228 - Configuration <WORKING_MODE> using the default value: WORKING_MODE.ONE_SHOT
2025-04-15 15:23:44.322 | INFO     | ASTRA.data_objects.DataClass:__init__:126 - DataClass opening 3 files from a list/tuple
2025-04-15 15:23:44.324 | INFO     | ASTRA.base_models.Frame:__init__:252 - Creating frame from: /home/amiguel/spectra_collection/ESPRESSO/proxima/r.ESPRE.2019-07-03T01:43:39.634_S2D_A.fits
2025-04-15 15:23:44.327 | WARNING  | ASTRA.Components.SpectrumComponent:regenerate_order_

After the model is constructed, we can remove the tellurics from all observations at once through

In [2]:
data.remove_telluric_features(ModelTell)


/home/amiguel/development/ASTRA/.venv/lib/python3.12/site-packages/pysynphot/locations.py:47: UserWarning: PYSYN_CDBS is undefined; functionality will be SEVERELY crippled.
  warnings.warn("PYSYN_CDBS is undefined; functionality will be SEVERELY "
/home/amiguel/development/ASTRA/.venv/lib/python3.12/site-packages/pysynphot/locations.py:346: UserWarning: Extinction files not found in extinction
  warnings.warn('Extinction files not found in %s' % (extdir, ))
/home/amiguel/development/ASTRA/.venv/lib/python3.12/site-packages/pysynphot/refs.py:117: UserWarning: No graph or component tables found; functionality will be SEVERELY crippled. No files found for http://ssb.stsci.edu/cdbs_open/cdbs/mtab/*_tmg.fits
  warnings.warn('No graph or component tables found; '
/home/amiguel/development/ASTRA/.venv/lib/python3.12/site-packages/pysynphot/refs.py:124: UserWarning: No thermal tables found, no thermal calculations can be performed. No files found for http://ssb.stsci.edu/cdbs_open/cdbs/mtab/*_